In [364]:
import numpy as np
import pandas as pd
import math
df=pd.read_csv('projdata.csv',encoding = "utf-8",engine='python',sep=",")


In [365]:
y=np.array(df[['Fault']])
X=df.as_matrix(columns=df.columns[1:])
X=X.T
y=y.T

C:\Users\GOGULAANAND\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [366]:
def sig(x):
    return 1/(1+np.exp(-x))

In [367]:
nx=np.min(np.shape(X))
ny=np.min(np.shape(y))
print(X.shape)
nh=3
nl=2
print("Size of input layer: "+str(nx))
print("Size of output layer: "+str(ny))
print("Size of hidden units in each layer: "+str(nh))
print("No of hidden layers: "+str(nl))


(3, 3287)
Size of input layer: 3
Size of output layer: 1
Size of hidden units in each layer: 3
No of hidden layers: 2


In [368]:
def initialise_params(nx,nh,ny):
    
    nh1=nh
    nh2=nh
    W1=np.random.randn(nh1,nx)*0.01
    b1=np.zeros((nh1,1))
    W2=np.random.randn(nh2,nh1)*0.01
    b2=np.zeros((nh2,1))
    W3=np.random.randn(ny,nh2)*0.01
    b3=np.zeros((ny,1))
    
    assert (W1.shape == (nh1, nx))
    assert (b1.shape == (nh1, 1))
    assert (W2.shape == (nh2, nh1))
    assert (b2.shape == (nh2, 1))
    assert (W3.shape == (ny, nh2))
    assert (b3.shape == (ny, 1))
    
    
    params={"W1":W1,"b1":b1,"W2":W2,"b2":b2,"W3":W3,"b3":b3}
    
    return params
    

In [369]:
params=initialise_params(nx,nh,ny)

In [370]:
def forward_prop(X,params):
    
    W1=params["W1"]
    b1=params["b1"]
    W2=params["W2"]
    b2=params["b2"]
    W3=params["W3"]
    b3=params["b3"]
    
    Z1=np.dot(W1,X)
    A1=np.tanh(Z1)
    Z2=np.dot(W2,A1)
    A2=np.tanh(Z2)
    Z3=np.dot(W3,A2)
    A3=sig(Z3)
    
    cache={"Z1":Z1,"A1":A1,"Z2":Z2,"A2":A2,"Z3":Z3,"A3":A3}
    
    return A3,cache

In [371]:
A3,cache=forward_prop(X,params)

In [372]:
def compute_cost(y,A3):
    
    m=np.max(y.shape)
        
    c1=np.dot(y,(np.log(A3)).T)
    c2=np.dot((1-y),(np.log(1-A3)).T)
    cost = -((c1+c2)/m)
    
    cost = np.squeeze(cost) 
    return (np.sum(cost))

In [373]:
cost=compute_cost(y,A3)
print(cost)

0.6931462450006023


In [374]:
def back_prop(y,params,cache,X):
    
    W1=params["W1"]
    b1=params["b1"]
    W2=params["W2"]
    b2=params["b2"]
    W3=params["W3"]
    b3=params["b3"]
    
    m=np.max(y.shape)
    
    A1=cache["A1"]
    A2=cache["A2"]
    A3=cache["A3"]
    
    
    dZ3 = A3-y
    dW3 = np.dot(dZ3,A3.T)/m
    db3 = np.sum(dZ3,axis=1,keepdims=True)/m
    dZ2 = np.dot(W3.T,dZ3)*(1-np.power(A2,2))
    dW2 = np.dot(dZ2,A2.T)/m
    db2 = np.sum(dZ2,axis=1,keepdims=True)/m
    dZ1 = np.dot(W2.T,dZ2)*(1-np.power(A1,2))
    dW1 = np.dot(dZ1,X.T)/m
    db1 = np.sum(dZ1,axis=1,keepdims=True)/m
    
    grads = {"dW1": dW1,
             "db1": db1,
             "dW2": dW2,
             "db2": db2,
             "dW3": dW3,
             "db3": db3}
    
    return grads


In [375]:
grads=back_prop(y,params,cache,X)

In [376]:
def update_params(grads,params,learning_rate=1.2):
    
    W1 = params["W1"]
    b1 = params["b1"]
    W2 = params["W2"]
    b2 = params["b2"]
    W3 = params["W3"]
    b3 = params["b3"]
    
    dW1 = grads["dW1"]
    db1 = grads["db1"]
    dW2 = grads["dW2"]
    db2 = grads["db2"]
    dW3 = grads["dW3"]
    db3 = grads["db3"]
    
    
    W1 = W1-(learning_rate*dW1)
    b1 = b1-(learning_rate*db1)
    W2 = W2-(learning_rate*dW2)
    b2 = b2-(learning_rate*db2)
    W3 = W3-(learning_rate*dW3)
    b3 = b3-(learning_rate*db3)
    
    
    params = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2,"W3":W3,"b3":b3
                }
    
    return params
    


In [377]:
learning_rate=0.1
params=update_params(grads,params,learning_rate)

In [378]:
def nn_model(X,y,nh,num_of_iters=5000,print_cost=False):
    
    nx=np.min(X.shape)
    ny=np.min(y.shape)
    
    parameters=initialise_params(nx,nh,ny)
    
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    W3 = parameters["W3"]
    b3 = parameters["b3"]
    learning_rate=0.01
    
    for i in range(0,num_of_iters,1):
        
        A3,cache=forward_prop(X,parameters)
        
        cost=compute_cost(y,A3)
        
        grads=back_prop(y,parameters,cache,X)
        
        parameters=update_params(grads,parameters,learning_rate)
        
        if print_cost and i % 1000 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
        
    return parameters

In [379]:
params=nn_model(X,y,nh,num_of_iters=5000,print_cost=True)

Cost after iteration 0: 0.693147
Cost after iteration 1000: 0.692952
Cost after iteration 2000: 0.690696
Cost after iteration 3000: 0.474053
Cost after iteration 4000: 0.253763


In [380]:
def predict(X,params):
    
    A3, cache = forward_prop(X,params)
    predictions = (A3>0.5)
    
    return predictions


In [381]:
predictions = predict(X,params)
print("predictions mean = " + str(np.mean(predictions)))

print ('Accuracy: %d' % float((np.dot(y,predictions.T) + np.dot(1-y,1-predictions.T))/float(y.size)*100) + '%')

predictions mean = 0.8414968055978096
Accuracy: 85%


In [ ]:
hidden_layer_sizes = [3, 4, 5, 20, 22, 50]
for i, nh in enumerate(hidden_layer_sizes):
    parameters = nn_model(X, y, nh, num_of_iters = 5000)
    predictions = predict(X,parameters)
    accuracy = float((np.dot(y,predictions.T) + np.dot(1-y,1-predictions.T))/float(y.size)*100)
    print ("Accuracy for {} hidden units: {} %".format(nh, accuracy))

Accuracy for 3 hidden units: 84.69729236385763 %
Accuracy for 4 hidden units: 85.57955582598113 %
Accuracy for 5 hidden units: 85.57955582598113 %
Accuracy for 20 hidden units: 86.21843626407059 %
